In [11]:
# dependencies
import pandas as pd
import requests
import json
import matplotlib.pyplot as plt
from citipy import citipy
import numpy as np
import time
import os

# establish openweather API key/url
file_name = "../../../PythonScripts/BootcampExercises/API_Keys/api_keys.json"
data = json.load(open(file_name))
key = data['openweather']
url = "http://api.openweathermap.org/data/2.5/weather?"
units = "imperial"
query_url = url + "appid=" + key + "&units=" + units + "&q="

# get olympic data files
csv_path = os.path.join("../Resources", "GDP_allyears.csv") #do not make changes
gdp = pd.read_csv(csv_path)
csv_path = os.path.join("../Resources", "country_participation.csv") #do not make changes
participation = pd.read_csv(csv_path)
csv_path = os.path.join("../Resources", "country_codes.csv") #do not make changes
codes = pd.read_csv(csv_path)

In [12]:
# join in country codes
participation = participation.merge(codes, on='Country', how='left')

# clean/format participation data set
participation["TotalMedals"] = participation["Gold"] + participation["Silver"] + participation["Bronze"]
participation = participation.fillna(0)
participation["From"] = participation["From"].astype(int)
participation["To"] = participation["To"].astype(int)
participation["Gold"] = participation["Gold"].astype(int)
participation["Silver"] = participation["Silver"].astype(int)
participation["Bronze"] = participation["Bronze"].astype(int)
participation["Sports"] = participation["Sports"].astype(int)
participation["TotalAthletes"] = participation["Participants"].astype(int)
participation = participation.drop('Participants', 1)
participation["TotalMedals"] = participation["TotalMedals"].astype(int)
participation = participation.rename(columns={'Sports': 'TotalSports','Gold':'TotalGold','Silver':'TotalSilver','Bronze':'TotalBronze'})

In [13]:
# determine number of participation years
gameyears = [1924, 1928, 1932, 1936, 1948, 1952, 1956, 1960, 1964, 1968, 1972, 1976, 1980, 1984, 1988, 1992, 1994, 1998, 2002, 2006, 2010, 2014]
for index, row in participation.iterrows():
    counter = 0
    To = row[2]
    From = row[1]
    for year in gameyears:
        if year >= From and year <= To:
            counter = counter+1
    participation.loc[index, "TotalYears"] = counter
participation["TotalYears"] = participation["TotalYears"].astype(int)

In [14]:
# find average medals per year per country
participation["AvgMedalsperYear"] = round(participation["TotalMedals"]/participation["TotalYears"],1)

# find average participants per year per country
participation["AvgAthleteperYear"] = round(participation["TotalAthletes"]/participation["TotalYears"],1)

# find average sports per year per country
participation["AvgSportsperYear"] = round(participation["TotalSports"]/participation["TotalYears"],1)

# find average participants per sport per country
participation["AvgAthleteperSport"] = round(participation["TotalAthletes"]/participation["TotalSports"],1)

In [15]:
participation.head()

,Country,From,To,TotalSports,TotalGold,TotalSilver,TotalBronze,Int Olympic Committee code,ISO code,TotalMedals,TotalAthletes,TotalYears,AvgMedalsperYear,AvgAthleteperYear,AvgSportsperYear,AvgAthleteperSport
0,Australia,1924,2014,14,5,3,4,AUS,AU,12,240,22,0.5,10.9,0.6,17.1
1,Austria,1924,2014,14,59,78,81,AUT,AT,218,859,22,9.9,39.0,0.6,61.4
2,Belgium,1924,2014,8,1,1,3,BEL,BE,5,124,22,0.2,5.6,0.4,15.5
3,Canada,1924,2014,15,62,56,52,CAN,CA,170,1279,22,7.7,58.1,0.7,85.3
4,Czechoslovakia,1924,1992,11,2,8,15,0,0,25,480,16,1.6,30.0,0.7,43.6


In [16]:
gdp

,COUNTRY,YEAR,USDperCapita
0,ARG,2004,12562.781240
1,ARG,2005,13995.318460
2,ARG,2006,15450.127910
3,ARG,2007,17139.476390
4,ARG,2008,18026.729750
5,ARG,2009,16940.242170
6,ARG,2010,18719.379470
7,ARG,2011,20076.691250
8,ARG,2012,20060.221770
9,ARG,2013,20693.539170
